I will try to implement every technique I know to clean up the data.

In [48]:
import pandas as pd
import os

from pandas import read_csv,read_json

output_path='data/datasets/'

Grab the data from the online repository

In [49]:
df_original=read_csv("https://raw.githubusercontent.com/Ikea-179/Hateful-Meme-Detection/main/data/new_merged.csv")

In [50]:
df_original.sample(10)

,Unnamed: 0.1,Unnamed: 0,id,Fulltext,text,img,label,clean text
2198,2198,2198,30697,the plate is so hot al gore is using it as evi...,the plate is so hot al gore is using it as evi...,img/30697.png,0,the plate is so hot al gore is using it as evi...
11394,11394,12594,2149,what's that burning? oh it's my pot roast Woma...,what's that burning? oh it's my pot roast,img/02149.png,0,whats that burning? oh its my pot roast Female...
6909,6909,6909,74218,grab her by the.. nevermind Woman Female White...,grab her by the.. nevermind,img/74218.png,1,grab her by the.. nevermindFemale WhiteCaitlyn...
4766,4766,4766,37295,how to get a black guy to see his baby Girl Gi...,how to get a black guy to see his baby,img/37295.png,1,how to get a black guy to see his baby Person ...
8697,8697,9117,82590,burning them was a great public service Man Ma...,burning them was a great public service,img/82590.png,1,burning them was a great public service Male M...
432,432,432,78560,"""so first of all you guys didn't even invite m...","""so first of all you guys didn't even invite m...",img/78560.png,0,"""so first of all you guys didnt even invite me..."
6653,6653,6653,98603,charles martel from the year 732 explains how ...,charles martel from the year 732 explains how ...,img/98603.png,1,charles martel from the year 732 explains how ...
4266,4266,4266,25370,porch monkey 4 life Man Man Man Male Male Male...,porch monkey 4 life,img/25370.png,0,porch monkey 4 lifeMale Male Male White White ...
4523,4523,4523,38215,they're not really after me they're after you ...,they're not really after me they're after you ...,img/38215.png,0,theyre not really after me theyre after you im...
4379,4379,4379,14523,every modern rap album cover Boy Person Person...,every modern rap album cover,img/14523.png,0,every modern rap album coverPerson Person Male...


All the dependencies

In [51]:
import nltk
import regex as re
import numpy as np
import autocorrect as ac        #https://github.com/filyp/autocorrect

from tqdm.auto import tqdm,trange  #https://tqdm.github.io
from autocorrect import Speller
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from gensim.parsing.preprocessing import remove_stopwords

#pd.options.mode.chained_assignment=None
min_string_len=12
max_string_len=200

stem=PorterStemmer()

In [52]:
def autocorrect_dataframe(df):
    """autocorrects the 'text' column and creates new column 'text_ac'. Input: Dataframe with 'text' column."""
    spell=Speller(lang='en',only_replacements=True) #only_replacements=True for OCR, otherwise false
    for i in trange(len(df),desc='Correcting Data'):
        df.at[i,'text_ac']=spell(df.at[i,'text'])
    return df

In [53]:
def clean_string(string): #function to clean a string | to be used in clean_csv
    string=string.lower() #convert string to lowercase
    string=re.sub('\n|\r|\t','',string) #cleaning of whitespaces
    string=re.sub(r'[^\w\s]+','',string) #cleaning of symbols
    return string

In [54]:
def stemming(string): #churi kora
    result = re.sub('[^a-zA-Z]',' ',string)
    result = result.lower()
    result = result.split()
    result = [stem.stem(word) for word in result if not word in stopwords.words('english')]
    result = ' '.join(result)
    return result

In [55]:
def clean_dataframe_with_ac(df): #cleaning the dataframe with autocorrection
    df=df.dropna() #drop all tuples with empty attributes
    #df['text']=df['title']+" "+df['text'] #append title to text
    length=[]
    [length.append(len(str(text))) for text in tqdm(df["text"],desc='Measuring length')] #calculate length of text string
    df['length']=length #saves the length of the string in new 'length' column
    df=df.drop(df['text'][df['length']<min_string_len].index,axis=0) #drop all tuples with less than min_string_len letters
    df=df.drop(df['text'][df['length']>max_string_len].index,axis=0) #drop all tuples with greater than max_string_len letters
    df.reset_index(drop=True,inplace=True) #reset all indices of the dataframe
    df=autocorrect_dataframe(df) #autocorrects the 'text' column and creates new column 'text_ac'
    vfunc=np.vectorize(clean_string) #vectorize the clean_string function for use in pandas df
    df['text_ac']=vfunc(df['text_ac']) #apply the vectorized function to further clean the data
    for i in trange(len(df),desc='Cleaning Data'):
        df.at[i,'text_s']=remove_stopwords(df.at[i,'text_ac']) #cleaning stopwords
        df.at[i,'text_ss']=stemming(df.at[i,'text_s']) #stemming, comment this line if accuracy drops
    length_ss=[]
    [length_ss.append(len(str(text))) for text in tqdm(df["text_ss"],desc='Measuring length')] #calculate length of an article
    df['length_ss']=length_ss #saves the length of the article in a new column
    df=df.drop(df['text_ss'][df['length_ss']<min_string_len].index,axis=0) #drop all tuples with less than min_string_len letters
    df=df.drop(df['text_ss'][df['length_ss']>max_string_len].index,axis=0) #drop all tuples with greater than max_string_len letters
    df=df.dropna() #drop all tuples with empty attributes
    df.reset_index(drop=True,inplace=True) #reset all indices of the dataframe
    return df

In [56]:
def clean_dataframe(df): #cleaning the dataframe
    df=df.dropna() #drop all tuples with empty attributes
    length=[]
    [length.append(len(str(text))) for text in tqdm(df["text"],desc='Measuring length')] #calculate length of an article
    df['length']=length #saves the length of the article in a new column
    df=df.drop(df['text'][df['length']<min_string_len].index,axis=0) #drop all tuples with less than min_string_len letters
    df=df.drop(df['text'][df['length']>max_string_len].index,axis=0) #drop all tuples with greater than max_string_len letters
    #df['text']=df['title']+" "+df['text'] #append title to text
    df.reset_index(drop=True,inplace=True) #reset all indexes of the dataset
    #df=autocorrect_dataframe(df)
    vfunc=np.vectorize(clean_string) #vectorize the clean_string function for use in pandas df
    df['text']=vfunc(df['text']) #apply the vectorized function to further clean the data
    for i in trange(len(df),desc='Cleaning Data'):
        df.at[i,'text_s']=remove_stopwords(df.at[i,'text']) #cleaning stopwords
        df.at[i,'text_ss']=stemming(df.at[i,'text_s']) #stemming, comment this line if accuracy drops
    length_ss=[]
    [length_ss.append(len(str(text))) for text in tqdm(df["text_ss"],desc='Measuring length')] #calculate length of an article
    df['length_ss']=length_ss #saves the length of the article in a new column
    df=df.drop(df['text_ss'][df['length_ss']<min_string_len].index,axis=0) #drop all cleaned tuples with less than min_string_len letters
    df=df.drop(df['text_ss'][df['length_ss']>max_string_len].index,axis=0) #drop all cleaned tuples with greater than max_string_len letters
    df=df.dropna() #drop all tuples with empty attributes
    df.reset_index(drop=True,inplace=True) #reset all indices of the dataframe
    return df

In [57]:
df_cleaned=clean_dataframe(df_original)
df_cleaned.sample(10)

Measuring length:   0%|          | 0/12140 [00:00<?, ?it/s]

Cleaning Data:   0%|          | 0/11945 [00:00<?, ?it/s]

Measuring length:   0%|          | 0/11945 [00:00<?, ?it/s]

,Unnamed: 0.1,Unnamed: 0,id,Fulltext,text,img,label,clean text,length,text_s,text_ss,length_ss
4295,4672,4672,35189,when you don't abort the baby and it dies anyw...,when you dont abort the baby and it dies anyways,img/35189.png,0,when you dont abort the baby and it dies anywa...,49,dont abort baby dies anyways,dont abort babi die anyway,26
2647,2896,2896,72594,this transgender shit is getting outta hand [...,this transgender shit is getting outta hand,img/72594.png,1,this transgender shit is getting outta handPic...,43,transgender shit getting outta hand,transgend shit get outta hand,29
1114,1213,1213,32674,a school bus that was engulfed in flames [['T...,a school bus that was engulfed in flames,img/32674.png,0,a school bus that was engulfed in flamesThomas...,40,school bus engulfed flames,school bu engulf flame,22
10591,11537,12737,13672,there's only one race is nascar! Boy Person Pe...,theres only one race is nascar,img/13672.png,0,theres only one race is nascar!Person Person F...,32,theres race nascar,there race nascar,17
4871,5303,5303,96083,we give your holy flag praise our lord and ens...,we give your holy flag praise our lord and ens...,img/96083.png,0,we give your holy flag praise our lord and ens...,51,holy flag praise lord enslaver,holi flag prais lord enslav,27
4273,4649,4649,58609,after group sex mohammed is very tired [['Ala...,after group sex mohammed is very tired,img/58609.png,1,"after group sex mohammed is very tiredAlamy, G...",38,group sex mohammed tired,group sex moham tire,20
3409,3718,3718,10639,"now i'm gonna use two fingers [['V sign', 'Ge...",now im gonna use two fingers,img/10639.png,0,"now im gonna use two fingersV sign, Gesture, v...",29,im gonna use fingers,im gonna use finger,19
10629,11583,12783,87095,how it feels to chew five gum Man Person Male ...,how it feels to chew five gum,img/87095.png,0,how it feels to chew five gumPerson Male Male ...,29,feels chew gum,feel chew gum,13
10683,11647,12847,61074,found this old photo of our grand parents on t...,found this old photo of our grand parents on t...,img/61074.png,0,found this old photo of our grand parents on t...,55,old photo grand parents quad,old photo grand parent quad,27
6705,7281,7281,60931,these video game armors are getting pretty rid...,these video game armors are getting pretty rid...,img/60931.png,0,these video game armors are getting pretty rid...,53,video game armors getting pretty ridiculous,video game armor get pretti ridicul,35


In [58]:
df_cleaned.tail(5)

,Unnamed: 0.1,Unnamed: 0,id,Fulltext,text,img,label,clean text,length,text_s,text_ss,length_ss
11124,12135,13335,71352,fighting for gay rights Woman Woman Woman Girl...,fighting for gay rights,img/71352.png,0,fighting for gay rights Female Female Female F...,23,fighting gay rights,fight gay right,15
11125,12136,13336,2164,that feeling when you finish your homework in ...,that feeling when you finish your homework in ...,img/02164.png,0,that feeling when you finish your homework in ...,86,feeling finish homework record time extra time...,feel finish homework record time extra time chill,49
11126,12137,13337,3587,the day that shook new york city [['9/11 Memo...,the day that shook new york city,img/03587.png,0,the day that shook new york city9/11 Memorial ...,32,day shook new york city,day shook new york citi,23
11127,12138,13338,47839,one of the first prototypes of the atom bomb ...,one of the first prototypes of the atom bomb,img/47839.png,0,one of the first prototypes of the atom bomb,44,prototypes atom bomb,prototyp atom bomb,18
11128,12139,13339,34281,today feels like a kinda day where i just wann...,today feels like a kinda day where i just wann...,img/34281.png,0,today feels like a kinda day where i just wann...,64,today feels like kinda day wanna freak people,today feel like kinda day wanna freak peopl,43


In [59]:
df_cleaned_ac=clean_dataframe_with_ac(df_original)
df_cleaned_ac.sample(10)

Measuring length:   0%|          | 0/12140 [00:00<?, ?it/s]

Correcting Data:   0%|          | 0/11945 [00:00<?, ?it/s]

Cleaning Data:   0%|          | 0/11945 [00:00<?, ?it/s]

Measuring length:   0%|          | 0/11945 [00:00<?, ?it/s]

,Unnamed: 0.1,Unnamed: 0,id,Fulltext,text,img,label,clean text,length,text_ac,text_s,text_ss,length_ss
10703,11685,12885,17590,loading the dishwasher in the back solid move ...,loading the dishwasher in the back solid move,img/17590.png,0,loading the dishwasher in the back solid moveD...,45,loading the dishwasher in the back solid move,loading dishwasher solid,load dishwash solid,19
6502,7069,7069,30782,an older gentlemen having a wonderful day [['...,an older gentlemen having a wonderful day,img/30782.png,0,an older gentlemen having a wonderful dayShelt...,41,an older gentlemen having a wonderful day,older gentlemen having wonderful day,older gentlemen wonder day,26
9021,9822,11022,62501,say no. it's not only the proper christian sta...,say no. it's not only the proper christian sta...,img/62501.png,0,say no. its not only the proper christian stan...,116,say no its not only the proper christian stanc...,proper christian stance christians duty god re...,proper christian stanc christian duti god resi...,63
5336,5805,5805,28516,anjem choudary wants this removed he said it i...,anjem choudary wants this removed he said it i...,img/28516.png,0,anjem choudary wants this removed he said it i...,94,anger choudary wants this removed he said it i...,anger choudary wants removed said insults isla...,anger choudari want remov said insult islam th...,62
7737,8418,8418,26385,islam is a cancer treat it with radiation [['...,islam is a cancer treat it with radiation,img/26385.png,1,islam is a cancer treat it with radiationNucle...,41,islam is a cancer treat it with radiation,islam cancer treat radiation,islam cancer treat radiat,25
10976,11984,13184,97246,he won because he is a boy competing in a fema...,he won because he is a boy competing in a fema...,img/97246.png,0,he won because he is a boy competing in a fema...,54,he won because he is a boy competing in a fema...,won boy competing female sport,boy compet femal sport,22
2079,2270,2270,52904,when you eat her ass and then she say that she...,when you eat her ass and then she say that she...,img/52904.png,0,when you eat her ass and then she say that she...,73,when you eat her ass and then she say that she...,eat ass dont suck dick payment,eat ass dont suck dick payment,30
5940,6453,6453,75421,mfw the local tranny gets lynched [['Shiba In...,mfw the local tranny gets lynched,img/75421.png,1,"mfw the local tranny gets lynchedShiba Inu, Ak...",33,mfd the local trains gets lynched,mfd local trains gets lynched,mfd local train get lynch,25
9136,9944,11144,42351,we are not here to be part of america we are h...,we are not here to be part of america we are h...,img/42351.png,1,we are not here to be part of america we are h...,70,we are not here to be part of america we are h...,america america,america america,15
2931,3206,3206,59124,i'm at the camp and i still can't concentrate ...,i'm at the camp and i still can't concentrate,img/59124.png,1,im at the camp and i still cant concentrateFem...,45,im at the camp and i still cant concentrate,im camp concentrate,im camp concentr,16


In [60]:
df_cleaned.loc[df_cleaned['length'].idxmax(axis=0,skipna=True)]

Unnamed: 0.1                                                 3850
Unnamed: 0                                                   3850
id                                                          60789
Fulltext        musslamic democrat, ilhan omar, has threatened...
text            musslamic democrat ilhan omar has threatened m...
img                                                 img/60789.png
label                                                           0
clean text      musslamic democrat, ilhan omar, has threatened...
length                                                        200
text_s          musslamic democrat ilhan omar threatened membe...
text_ss         musslam democrat ilhan omar threaten member co...
length_ss                                                     117
Name: 3532, dtype: object

In [61]:
df_cleaned.loc[df_cleaned['length'].idxmin(axis=0,skipna=True)]

Unnamed: 0.1                                                   97
Unnamed: 0                                                     97
id                                                          30247
Fulltext        broken chair  [['Chair', 'Office chair', 'Furn...
text                                                 broken chair
img                                                 img/30247.png
label                                                           0
clean text      broken chairChair, Office chair, Furniture, Ta...
length                                                         12
text_s                                               broken chair
text_ss                                              broken chair
length_ss                                                      12
Name: 82, dtype: object

In [62]:
df_cleaned_2=df_cleaned[['id','text','text_s','text_ss','label']]
df_cleaned_2.sample(10)

,id,text,text_s,text_ss,label
1338,19543,whats the difference between boy scouts and je...,whats difference boy scouts jews boy scouts co...,what differ boy scout jew boy scout come camp,1
8689,73842,international boxing day,international boxing day,intern box day,1
11107,70198,president condemned capital punishment,president condemned capital punishment,presid condemn capit punish,0
8495,65917,this is what science does for humanity this is...,science humanity islam humanity,scienc human islam human,1
9312,68749,my name is osama bin laden and this is jackass,osama bin laden jackass,osama bin laden jackass,1
2696,10893,marge i converted to islam,marge converted islam,marg convert islam,1
1874,35091,with black power comes no responsibility,black power comes responsibility,black power come respons,1
130,79856,my friends coming to see me after a crazy frid...,friends coming crazy friday night,friend come crazi friday night,0
7742,25830,i wanna destroy radical islam will you help me...,wanna destroy radical islam help yes hell yeah,wanna destroy radic islam help ye hell yeah,1
10536,42689,this happens to 13 of people 23 arent doing this,happens 13 people 23 arent,happen peopl arent,0


In [63]:
df_cleaned_ac_2=df_cleaned_ac[['id','text','text_ac','text_s','text_ss','label']]
df_cleaned_ac_2.sample(10)

,id,text,text_ac,text_s,text_ss,label
10065,5694,represent your country's flag until death,represent your countrys flag until death,represent countrys flag death,repres countri flag death,0
2662,72406,speak out to end domestic violence,speak out to end domestic violence,speak end domestic violence,speak end domest violenc,0
8004,6723,the hardest part to eat in a vegetable,the hardest part to eat in a vegetable,hardest eat vegetable,hardest eat veget,1
10505,20831,shia men are hungry men,shia men are hungry men,shia men hungry men,shia men hungri men,0
2535,46827,"""those that attack one muslim, are attacking a...",those that attack one muslim are attacking all...,attack muslim attacking muslims attack america...,attack muslim attack muslim attack american at...,0
10393,38652,claims that her ancestors fought an enemy thes...,claims that her ancestors fought an enemy thes...,claims ancestors fought enemy enemies,claim ancestor fought enemi enemi,0
4305,21604,bruce jenner's cat,bruce jenners cat,bruce jenners cat,bruce jenner cat,0
6636,96583,for just ten cents a day you can help feed the...,for just ten cents a day you can help feed the...,cents day help feed children,cent day help feed children,0
4574,28731,how africans eat their food mmmm....... delicious,how africana eat their food mmmm delicious,africana eat food mmmm delicious,africana eat food mmmm delici,1
6782,27083,never wear gloves when operating rotating mach...,never wear gloves when operating rotating mach...,wear gloves operating rotating machinery regard,wear glove oper rotat machineri regard,1


In [64]:

if os.path.isdir(output_path):
    df_cleaned_2.to_csv(output_path+'df_cleaned_2.csv')
    df_cleaned_ac_2.to_csv(output_path+'df_cleaned_ac_2.csv')
else:
    os.mkdir(output_path)
    df_cleaned_2.to_csv(output_path+'df_cleaned_2.csv')
    df_cleaned_ac_2.to_csv(output_path+'df_cleaned_ac_2.csv')